In [7]:
from sklearn.neighbors import KNeighborsClassifier

import numpy as np

# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

database_path = '/workspace/data/crops_and_embeddings/kwitonda_database/embeddings'
database_embeddings = np.load(f'{database_path}/embeddings_dino.npy')
database_labels = np.load(f'{database_path}/individual_ids.npy')

path = '/workspace/data/crops_and_embeddings/cameratrap_02_2024/embeddings'

embeddings = np.load(f'{path}/embeddings_dino.npy')
sequence_ids = np.load(f'{path}/sequence_ids.npy')
labels = np.load(f'{path}/individual_ids.npy')

# average all embeddings for each sequence

unique_sequence_ids = np.unique(sequence_ids)

new_embeddings = []
new_labels = []

for sequence_id in unique_sequence_ids:
    mask = sequence_ids == sequence_id
    new_embeddings.append(embeddings[mask].mean(axis=0))
    new_labels.append(labels[mask][0])

# embeddings = np.array(new_embed?dings)
# labels = np.array(new_labels)


assert len(embeddings) == len(labels)

print(embeddings.shape)
print(labels.shape)

(5737, 1536)
(5737,)


In [8]:
def predict_one(embeddings, labels, database_embeddings, database_labels, test_idx, k=5):
    train_idx = [i for i in range(len(embeddings)) if i != test_idx]
    train_embeddings = database_embeddings
    train_labels = database_labels

    test_embedding = embeddings[test_idx]

    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(train_embeddings, train_labels)

    return knn.predict([test_embedding])[0], labels[test_idx]

# predict_one(embeddings, labels, 3)

# Calculate precision, recall, f1

def calculate_metrics(embeddings, labels, database_embeddings, database_labels, min_samples=5, k=2):
    # only predicting on individuals with at least min_samples
    unique_labels, counts = np.unique(labels, return_counts=True)
    unique_labels = unique_labels[counts >= min_samples]
    
    mask = np.isin(labels, unique_labels)
    # embeddings = embeddings[mask]
    # labels = labels[mask]
    
    idx = [i for i in range(len(embeddings)) if labels[i] in unique_labels]
    
    print(f'Calculating metrics on {len(idx)} samples from {len(unique_labels)} individuals with {len(embeddings)} samples in total.')
    
    predictions = [predict_one(embeddings, labels, database_embeddings, database_labels, i, k=k) for i in idx]
    predictions = np.array(predictions)
    
    tp = np.sum(predictions[:, 0] == predictions[:, 1])
    
    average_precision =  tp / len(embeddings)
    
    return average_precision
 

print(calculate_metrics(embeddings, labels, database_embeddings, database_labels, min_samples=1, k=1))
print(calculate_metrics(embeddings, labels, database_embeddings, database_labels, min_samples=5, k=3))

Calculating metrics on 5737 samples from 15 individuals with 5737 samples in total.
0.03712741851141712
Calculating metrics on 5734 samples from 14 individuals with 5737 samples in total.
0.02039393411190518
